In [1]:
import pandas as pd
import numpy as np
import optuna
import xgboost as xgb
import tensorflow as tf
from scipy.signal import butter, filtfilt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import zipfile
import tempfile
import os

# Suppress warnings
optuna.logging.set_verbosity(optuna.logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ==========================================
# 1. ADAPTIVE PIPELINE (Handles Mixed Hz)
# ==========================================
class AdaptiveAccelPipeline:
    def __init__(self, df):
        self.df = df.copy()
        self.df['local_ts'] = pd.to_datetime(self.df['local_ts'])
        # Sort is critical for time-series
        self.df = self.df.sort_values(by=['subject', 'local_ts']).reset_index(drop=True)

    def apply_adaptive_filter(self, target_cutoff_hz=5.0):
        """
        Applies filtering subject-by-subject.
        If a subject's sampling rate is too low, it auto-adjusts or skips filtering.
        """
        print(f"--- Running Adaptive Filter (Target Cutoff: {target_cutoff_hz}Hz) ---")
        
        def filter_subject_data(group):
            # 1. Calculate Sampling Rate (fs) for THIS subject
            # We filter out 0.0 diffs (duplicates) to avoid divide-by-zero
            time_diffs = group['local_ts'].diff().dt.total_seconds()
            valid_diffs = time_diffs[time_diffs > 0]
            
            if len(valid_diffs) < 10:
                return group # Too little data to process
            
            median_diff = valid_diffs.median()
            if median_diff == 0: return group # Safety catch
            
            fs = 1 / median_diff
            nyquist = 0.5 * fs
            
            # 2. DECISION LOGIC
            # Case A: Data is too slow for the requested filter (e.g., Data is 5Hz, you want 5Hz cut)
            if fs <= target_cutoff_hz * 2.0:
                # If data is really slow (<10Hz), usually we just skip filtering 
                # because it's already "smooth" compared to 20Hz data.
                # Or we apply a very mild smoothing (e.g. 0.8 * Nyquist)
                actual_cutoff = nyquist * 0.9 
                
                # If the resulting cutoff is tiny, just skip it.
                if actual_cutoff < 1.0:
                    return group 
            else:
                # Case B: Data is fast enough (e.g., 20Hz data, 5Hz cut) -> Use requested cutoff
                actual_cutoff = target_cutoff_hz

            # 3. Apply Filter
            try:
                b, a = butter(N=4, Wn=actual_cutoff / nyquist, btype='low')
                # Apply to X, Y, Z
                for col in ['x', 'y', 'z']:
                    group[col] = filtfilt(b, a, group[col])
            except Exception as e:
                # If math fails (e.g., unstable IIR), return raw data
                pass
                
            return group

        # Apply the logic to each subject independently
        self.df = self.df.groupby('subject', group_keys=False).apply(filter_subject_data)
        print("--- Adaptive Filtering Complete ---")
        return self.df

    def convert_to_gravity(self):
        print("--- Converting to Gravity & ENMO ---")
        scale = 16384.0 # Adjust based on your accelerometer range (e.g. +/- 2g vs +/- 4g)
        self.df['x_g'] = self.df['x'] / scale
        self.df['y_g'] = self.df['y'] / scale
        self.df['z_g'] = self.df['z'] / scale
        self.df['mag'] = np.sqrt(self.df['x_g']**2 + self.df['y_g']**2 + self.df['z_g']**2)
        self.df['enmo'] = np.maximum(self.df['mag'] - 1, 0)
        return self.df

    def calc_odba(self):
        # Simplified ODBA calculation
        self.df['odba'] = (self.df['x_g'] - self.df['x_g'].mean()).abs() + \
                          (self.df['y_g'] - self.df['y_g'].mean()).abs() + \
                          (self.df['z_g'] - self.df['z_g'].mean()).abs()
        return self.df

    def resample_and_label(self, interval_seconds=10, coherence_threshold=0.7):
        """
        Resamples data into windows. 
        ASSIGN LABELS based on threshold:
        If > 70% of the raw samples in the window are 'Grazing', the window is 'Grazing'.
        Otherwise, the window is discarded (Ambiguous).
        """
        print(f"--- Resampling ({interval_seconds}s) with Threshold {coherence_threshold*100}% ---")
        
        # Custom Aggregator for Labels
        def threshold_labeler(x):
            if x.empty: return np.nan
            counts = x.value_counts(normalize=True)
            # Check if the most frequent label crosses the threshold
            if counts.iloc[0] >= coherence_threshold:
                return counts.index[0]
            return np.nan # Drop this window (too messy/transitioning)

        # Feature Aggregators
        agg_dict = {
            'x_g': ['mean', 'std', 'min', 'max'],
            'y_g': ['mean', 'std', 'min', 'max'],
            'z_g': ['mean', 'std', 'min', 'max'],
            'mag': ['mean', 'std'],      
            'enmo': ['mean', 'max'], 
            'odba': ['mean', 'std'],    
            'behavioral_category': threshold_labeler # <--- LOGIC APPLIED HERE
        }

        resampled = (
            self.df.set_index('local_ts')
            .groupby('subject')
            .resample(f'{interval_seconds}s')
            .agg(agg_dict)
        )
        
        # Flatten columns
        resampled.columns = [f"{c[0]}_{c[1]}" if c[1] else c[0] for c in resampled.columns]
        resampled = resampled.rename(columns={'behavioral_category_threshold_labeler': 'label'})
        
        # DROP windows that failed the threshold check
        final_df = resampled.dropna(subset=['label']).reset_index()
        
        print(f"Generated {len(final_df)} labeled windows.")
        return final_df

    def create_time_series_sequences(self, data, feature_cols, target_col, time_steps=5):
        """
        Converts the resampled windows into sequences for LSTM.
        Input: (N, Features) -> Output: (N, TimeSteps, Features)
        """
        X_seq, y_seq = [], []
        
        # Group by subject to prevent sequences bleeding across different animals
        for subject, group in data.groupby('subject'):
            group = group.sort_values('local_ts')
            feats = group[feature_cols].values
            targets = group[target_col].values
            
            if len(group) < time_steps: continue
            
            # Sliding window over the WINDOWS
            for i in range(len(group) - time_steps):
                X_seq.append(feats[i : i + time_steps])
                y_seq.append(targets[i + time_steps]) # Predict the label of the *next* window
                
        return np.array(X_seq), np.array(y_seq)

# ==========================================
# 2. DEEP LEARNING ENGINE
# ==========================================
class TimeSeriesModeler:
    def __init__(self, pipeline, df):
        self.pipeline = pipeline
        self.df = df
        self.le = LabelEncoder()
        
    def run(self, time_steps=5):
        print(f"\n>>> Running Time Series Model (Lookback: {time_steps} steps)")
        
        # 1. Identify Feature Columns (exclude metadata)
        feature_cols = [c for c in self.df.columns if c not in ['subject', 'local_ts', 'label']]
        
        # 2. Encode Labels
        self.df['label_encoded'] = self.le.fit_transform(self.df['label'])
        num_classes = len(self.le.classes_)
        
        # 3. Scale Features (Crucial for LSTM)
        scaler = StandardScaler()
        # We perform scaling on the dataframe before sequencing
        scaled_df = self.df.copy()
        scaled_df[feature_cols] = scaler.fit_transform(scaled_df[feature_cols])
        
        # 4. Generate Sequences
        X, y = self.pipeline.create_time_series_sequences(
            scaled_df, feature_cols, 'label_encoded', time_steps
        )
        
        if len(X) == 0:
            print("Not enough data for sequences.")
            return None

        # 5. One-Hot Encode Targets
        y_cat = to_categorical(y, num_classes=num_classes)
        
        # 6. Train/Test Split
        X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)
        
        # 7. Define LSTM Model
        model = Sequential([
            Input(shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(64, return_sequences=False),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
        
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        # 8. Train
        es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        history = model.fit(
            X_train, y_train, 
            validation_data=(X_test, y_test),
            epochs=20, 
            batch_size=32, 
            callbacks=[es],
            verbose=0 # Silent training
        )
        
        # 9. Evaluate
        y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
        y_true = np.argmax(y_test, axis=1)
        
        acc = accuracy_score(y_true, y_pred)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
        
        return {
            'Accuracy': acc,
            'F1_Score': f1,
            'Time_Steps': time_steps,
            'Num_Training_Sequences': len(X_train)
            
        }

# ==========================================
# 3. MAIN EXECUTION
# ==========================================

# A. LOAD DATA
actual_file = []
main_path = "/home/rajesh/work/acclerometer_project/zip_data"

if os.path.exists(main_path):
    for zip_file in os.listdir(main_path):
        if zip_file.endswith(".zip"):
            with tempfile.TemporaryDirectory() as temp_dir:
                try:
                    with zipfile.ZipFile(os.path.join(main_path, zip_file), "r") as zf:
                        zf.extractall(temp_dir)
                        for root, dirs, files in os.walk(temp_dir):
                            for d in dirs:
                                if d.startswith('Processed'):
                                    sec_path = os.path.join(root, d)
                                    for f in os.listdir(sec_path):
                                        if f.endswith(('.xls', '.xlsx')):
                                            actual_file.append(pd.read_excel(os.path.join(sec_path, f)))
                except Exception as e: print(f"Error: {e}")
    if actual_file: df_raw = pd.concat(actual_file)
    else: df_raw = pd.DataFrame() # Handle empty case
else:
    # Dummy data for testing if path doesn't exist
    print("Using Dummy Data")
    dates = pd.date_range('2023-01-01', periods=5000, freq='200ms') # 5Hz data
    df_raw = pd.DataFrame({
        'subject': ['Cow1']*5000,
        'local_ts': dates,
        'x': np.random.randn(5000)*1000, 
        'y': np.random.randn(5000)*1000,
        'z': np.random.randn(5000)*1000,
        'behavioral_category': ['Grazing']*5000
    })

if not df_raw.empty:
    # B. INITIALIZE ADAPTIVE PIPELINE
    pipeline = AdaptiveAccelPipeline(df_raw)

    # 1. Adaptively Filter (safe for mixed 5Hz/10Hz/20Hz)
    pipeline.apply_adaptive_filter(target_cutoff_hz=5.0)

    # 2. Physics conversions
    pipeline.convert_to_gravity()
    pipeline.calc_odba()

    results_table = []
    
    # Grid Search Settings
    intervals = [10, 30]  # Window sizes in seconds
    thresholds = [0.6, 0.8] # Strictness of labeling (60% vs 80% purity)
    lstm_steps = [5, 10]    # How far back the LSTM looks

    print(f"\n--- Starting Grid Search ---")
    for iv in intervals:
        for th in thresholds:
            # 3. Resample and Label
            # This applies the logic: "Only keep window if > X% of data matches"
            df_ready = pipeline.resample_and_label(interval_seconds=iv, coherence_threshold=th)
            
            if len(df_ready) < 100:
                print(f"Skipping Interval={iv}, Threshold={th} (Not enough valid windows)")
                continue

            # 4. Run Time Series Model
            ts_modeler = TimeSeriesModeler(pipeline, df_ready)
            
            for steps in lstm_steps:
                res = ts_modeler.run(time_steps=steps)
                if res:
                    res['Window_Size_Sec'] = iv
                    res['Label_Threshold'] = th
                    results_table.append(res)

    # C. SAVE RESULTS
    if results_table:
        final_df = pd.DataFrame(results_table)
        final_df = final_df.sort_values(by='F1_Score', ascending=False)
        
        print("\n=== FINAL RESULTS ===")
        print(final_df)
        
        save_path = '/home/rajesh/work/acclerometer_project/codes/timeseries_results.csv'
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        final_df.to_csv(save_path, index=False)
    else:
        print("No results generated.")
else:
    print("No Data Loaded.")

2025-12-14 08:34:17.640866: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-14 08:34:17.668562: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-14 08:34:18.544816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-14 08:34:20.740447: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

KeyboardInterrupt: 

In [9]:
final_df

,Accuracy,F1_Score,Time_Steps,Num_Training_Sequences,Window_Size_Sec,Label_Threshold
3,0.929608,0.928017,10,14888,10,0.8
1,0.915121,0.913842,10,15736,10,0.6
7,0.904425,0.900703,10,4516,30,0.8
5,0.877276,0.870692,10,5049,30,0.6
2,0.857603,0.852966,5,14916,10,0.8
6,0.847845,0.842795,5,4544,30,0.8
0,0.831304,0.824163,5,15764,10,0.6
4,0.829134,0.821685,5,5077,30,0.6
